In [1]:
import pickle
import pandas as pd
import datetime

In [2]:
train_startdate = datetime.datetime.strptime('2019-01-01', "%Y-%m-%d").date()
train_enddate = datetime.datetime.strptime('2020-01-01', "%Y-%m-%d").date()

test_startdate = datetime.datetime.strptime('2020-01-01', "%Y-%m-%d").date()
test_enddate = datetime.datetime.strptime('2021-01-01', "%Y-%m-%d").date()

In [3]:
with open('./players.pkl', 'rb') as file:
    players = pickle.load(file)
    players = pd.DataFrame(players.values())

with open('./results.pkl', 'rb') as file:
    results = pickle.load(file)

with open('./tournaments.pkl', 'rb') as file:
    tournaments = pickle.load(file)
    tournaments = pd.DataFrame(tournaments.values())

In [4]:
results_df = pd.DataFrame([team_info | {"res_from": key} for key, teams in results.items() for team_info in teams])

In [36]:
notnull_mask = ~results_df['mask'].isnull()
nonnull_res = results_df[notnull_mask]
nonnull_res.head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,res_from
2407,"{'id': 1, 'name': 'Неспроста', 'town': {'id': ...",0111011101101110001101110011111111110011111100...,"{'name': 'КП - Неспроста', 'town': {'id': 201,...",67.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2408,"{'id': 2, 'name': 'Афина', 'town': {'id': 201,...",0111111101011010010101110111111111110011011111...,"{'name': 'Афина', 'town': {'id': 201, 'name': ...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2409,"{'id': 670, 'name': 'Ксеп', 'town': {'id': 201...",0011111101011010011101110011111111110111111111...,"{'name': 'Ксеп', 'town': {'id': 201, 'name': '...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2410,"{'id': 173, 'name': 'ЮМА', 'town': {'id': 285,...",0111111001101110001101111011111111110100111111...,"{'name': 'ЮМА-Энергокапитал', 'town': {'id': 2...",64.0,None,4.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2411,"{'id': 175, 'name': 'Транссфера', 'town': {'id...",0111111001101111001111111011111110110010111110...,"{'name': 'Транссфера', 'town': {'id': 285, 'na...",64.0,None,4.5,[],[],[],22


In [46]:
nonnull_res.groupby(by='res_from').apply(lambda x: max(x['questionsTotal'])).sum()

136785.0

In [44]:
nonnull_res[nonnull_res['res_from'] == 22].head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,res_from
2407,"{'id': 1, 'name': 'Неспроста', 'town': {'id': ...",0111011101101110001101110011111111110011111100...,"{'name': 'КП - Неспроста', 'town': {'id': 201,...",67.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2408,"{'id': 2, 'name': 'Афина', 'town': {'id': 201,...",0111111101011010010101110111111111110011011111...,"{'name': 'Афина', 'town': {'id': 201, 'name': ...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2409,"{'id': 670, 'name': 'Ксеп', 'town': {'id': 201...",0011111101011010011101110011111111110111111111...,"{'name': 'Ксеп', 'town': {'id': 201, 'name': '...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2410,"{'id': 173, 'name': 'ЮМА', 'town': {'id': 285,...",0111111001101110001101111011111111110100111111...,"{'name': 'ЮМА-Энергокапитал', 'town': {'id': 2...",64.0,None,4.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2411,"{'id': 175, 'name': 'Транссфера', 'town': {'id...",0111111001101111001111111011111110110010111110...,"{'name': 'Транссфера', 'town': {'id': 285, 'na...",64.0,None,4.5,[],[],[],22


In [6]:
tournaments['dateStart'] = pd.to_datetime(
    tournaments['dateStart'],
    infer_datetime_format=True,
    utc = True).dt.date
tournaments['dateEnd'] = pd.to_datetime(
    tournaments['dateEnd'],
    infer_datetime_format=True,
    utc = True).dt.date

In [35]:
mask = (train_startdate <= tournaments['dateStart']) & (tournaments['dateEnd'] < train_enddate)
tournaments[mask].head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
3921,4772,Синхрон северных стран. Зимний выпуск,2019-01-05,2019-01-09,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4115,4973,Балтийский Берег. 3 игра,2019-01-25,2019-01-29,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4116,4974,Балтийский Берег. 4 игра,2019-03-01,2019-03-05,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4117,4975,Балтийский Берег. 5 игра,2019-04-05,2019-04-09,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4128,4986,ОВСЧ. 6 этап,2019-02-15,2019-02-19,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}"


,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,res_from
2407,"{'id': 1, 'name': 'Неспроста', 'town': {'id': ...",0111011101101110001101110011111111110011111100...,"{'name': 'КП - Неспроста', 'town': {'id': 201,...",67.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2408,"{'id': 2, 'name': 'Афина', 'town': {'id': 201,...",0111111101011010010101110111111111110011011111...,"{'name': 'Афина', 'town': {'id': 201, 'name': ...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2409,"{'id': 670, 'name': 'Ксеп', 'town': {'id': 201...",0011111101011010011101110011111111110111111111...,"{'name': 'Ксеп', 'town': {'id': 201, 'name': '...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2410,"{'id': 173, 'name': 'ЮМА', 'town': {'id': 285,...",0111111001101110001101111011111111110100111111...,"{'name': 'ЮМА-Энергокапитал', 'town': {'id': 2...",64.0,None,4.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2411,"{'id': 175, 'name': 'Транссфера', 'town': {'id...",0111111001101111001111111011111110110010111110...,"{'name': 'Транссфера', 'town': {'id': 285, 'na...",64.0,None,4.5,[],[],[],22


In [33]:
trainset = nonnull_res[nonnull_res['res_from'].isin(tournaments[mask]['id'])]
trainset.head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,res_from
366207,"{'id': 45556, 'name': 'Рабочее название', 'tow...",111111111011111110111111111100010010,"{'name': 'Рабочее название', 'town': {'id': 28...",28.0,"{'id': 56392, 'venue': {'id': 3030, 'name': 'С...",1.0,"[{'id': 91169, 'questionNumber': 15, 'answer':...",[],"[{'flag': 'Б', 'usedRating': 13507, 'rating': ...",4772
366208,"{'id': 1030, 'name': 'Сборная Бутана', 'town':...",111111111011110100101111011001011010,"{'name': 'Сборная Бутана', 'town': {'id': 346,...",25.0,"{'id': 56690, 'venue': {'id': 3151, 'name': 'У...",5.5,[],[],"[{'flag': None, 'usedRating': 13058, 'rating':...",4772
366209,"{'id': 4252, 'name': 'Ять', 'town': {'id': 197...",111111111011110101101111001011110000,"{'name': 'Ять', 'town': {'id': 197, 'name': 'М...",25.0,"{'id': 56814, 'venue': {'id': 3112, 'name': 'М...",5.5,"[{'id': 91164, 'questionNumber': 33, 'answer':...",[],"[{'flag': 'К', 'usedRating': 9584, 'rating': 9...",4772
366210,"{'id': 5444, 'name': 'Эйфью', 'town': {'id': 1...",101111101111111110001101011001111010,"{'name': 'Эйфью', 'town': {'id': 197, 'name': ...",25.0,"{'id': 56814, 'venue': {'id': 3112, 'name': 'М...",5.5,[],[],"[{'flag': 'Л', 'usedRating': 8592, 'rating': 8...",4772
366211,"{'id': 40931, 'name': 'Здоровенный Я', 'town':...",111111101011111101000111001001111110,"{'name': 'Здоровенный Я', 'town': {'id': 201, ...",25.0,"{'id': 55460, 'venue': {'id': 3117, 'name': 'М...",5.5,[],[],"[{'flag': 'Л', 'usedRating': 12069, 'rating': ...",4772


После некоторых раздумий, предлагаем следующие фичи:

    - Каждый игрок - отдельная фича
    - Каждый встреченный в трейн-тесте вопрос - отдельная фича

В качестве таргета мы рассматриваем bool - ответила команда на вопрос или нет.

При таком подходе мы можем после обучения взять веса при каждой фиче игрока и смотреть на неё, как на силу этого игрока. Отметим, что из такой модели можно вытащить сложность вопросов в трейн выборке (вес для соответствующей фичи). Однако, предсказания на трейн сете мы делать не сможем, поскольку вопросы там (скорее всего) будут другие, да и установить тот же это вопрос или нет мы не можем.

С учётом страшности наших данных (файл `results.pkl` довольно долго грузился и неслабо заполнял память), мы считаем важным оценить, сколько у нас есть игроков и вопросов, поскольку мы по сути хотим получить OHE-представления вопросов и игроков.

#### Количество вопросов:

In [52]:
trainset.groupby('res_from').apply(lambda x: max(x['mask'].apply(len))).sum()

31320

#### Количество игроков:

In [56]:
trainset['teamMembers'].iloc[0]

[{'flag': 'Б',
  'usedRating': 13507,
  'rating': 13507,
  'player': {'id': 6212,
   'name': 'Юрий',
   'patronymic': 'Яковлевич',
   'surname': 'Выменец'}},
 {'flag': 'Б',
  'usedRating': 10988,
  'rating': 13185,
  'player': {'id': 18332,
   'name': 'Александр',
   'patronymic': 'Витальевич',
   'surname': 'Либер'}},
 {'flag': 'Б',
  'usedRating': 8534,
  'rating': 12801,
  'player': {'id': 18036,
   'name': 'Михаил',
   'patronymic': 'Ильич',
   'surname': 'Левандовский'}},
 {'flag': 'К',
  'usedRating': 6401,
  'rating': 12801,
  'player': {'id': 22799,
   'name': 'Сергей',
   'patronymic': 'Игоревич',
   'surname': 'Николенко'}},
 {'flag': 'Б',
  'usedRating': 4252,
  'rating': 12757,
  'player': {'id': 15456,
   'name': 'Сергей',
   'patronymic': 'Владимирович',
   'surname': 'Коновалов'}},
 {'flag': 'Б',
  'usedRating': 2069,
  'rating': 12416,
  'player': {'id': 26089,
   'name': 'Ирина',
   'patronymic': 'Сергеевна',
   'surname': 'Прокофьева'}}]

In [120]:
player_ids_lst = [user_info['player']['id'] for team_info in trainset['teamMembers'] for user_info in team_info]
player_ids = {idx: id_ for idx, id_ in enumerate(set(player_ids_lst))}

In [122]:
len(player_ids_lst), len(player_ids)

(398135, 52871)

Выходит что-то около 84000. В целом приемлемо (с учётом количества точек данных, увидим это чуть ниже).

In [70]:
trainset = trainset.assign(playerIds=[[user_info['player']['id'] for user_info in team_info] for team_info in trainset['teamMembers']])

In [87]:
train_dataset = trainset[['mask', 'res_from', 'playerIds']]
train_dataset.sample(5)

,mask,res_from,playerIds
489245,1110011101111111011110111X0110011011,5819,"[8532, 19992, 44650, 53426]"
462529,011110100010000001000000000100110000,5698,"[203422, 203423, 203424, 203425, 203426, 203427]"
472150,111111110101110111111000101000011011,5740,"[31983, 102114, 68433, 38846, 79135]"
438264,110000000001110001100000010000011010,5402,"[103111, 144665, 130863, 4733, 20114, 203855]"
409748,111001100010011000010001101100100001,5071,"[88112, 121709, 94091, 17057, 146382, 14230]"


In [111]:
grouped = train_dataset.groupby('res_from', group_keys=True).apply(lambda x: x[['mask', 'playerIds']])
grouped.head()

mask  \
res_from                                                
4772     366207  111111111011111110111111111100010010   
         366208  111111111011110100101111011001011010   
         366209  111111111011110101101111001011110000   
         366210  101111101111111110001101011001111010   
         366211  111111101011111101000111001001111110   

                                                  playerIds  
res_from                                                     
4772     366207   [6212, 18332, 18036, 22799, 15456, 26089]  
         366208           [1585, 40840, 1584, 10998, 16206]  
         366209  [23513, 18168, 21060, 35850, 31332, 10187]  
         366210         [36742, 28939, 54289, 15381, 27375]  
         366211  [28689, 17720, 30597, 12400, 26988, 69476]

Создадим табличку, в которой будем хранить OHE-векторы

In [115]:
question_id = 0
for res_from, data in train_dataset.groupby('res_from'):
    # Here we have a group of teams from tournament
    
    for index, row in data.iterrows():
        # Here we have mask and ids
        new_row = []
        
        for i, has_answered in enumerate(row['mask']):
            
    
    
    
    num_questions = len(data['mask'].iloc[0])
    

                                        mask  res_from  \
366207  111111111011111110111111111100010010      4772   
366208  111111111011110100101111011001011010      4772   
366209  111111111011110101101111001011110000      4772   
366210  101111101111111110001101011001111010      4772   
366211  111111101011111101000111001001111110      4772   
...                                      ...       ...   
366433  110000000001110000000000000000100000      4772   
366434  100010100001100000001000000000000000      4772   
366435  101010000000100000001000001000000000      4772   
366436  100000000000100100001000000001100000      4772   
366437  100000100000100000001000000000000000      4772   

                                               playerIds  
366207         [6212, 18332, 18036, 22799, 15456, 26089]  
366208                 [1585, 40840, 1584, 10998, 16206]  
366209        [23513, 18168, 21060, 35850, 31332, 10187]  
366210               [36742, 28939, 54289, 15381, 27375]  
366211  

In [12]:
len(results_df['res_from'].unique())

5277

In [85]:
len(tournaments), len(results)

(5528, 5528)